In [125]:
# Import needed libraries
import numpy as np
from joblib import dump, load
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsOneClassifier
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import sklearn.cluster
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest 
from functions import numeric_preproccessing, low_pass_filter_anomaly_detection, augment_features,augment_features_gradient,augment_features_window

In [3]:
data = pd.read_csv('imputed.csv')
data

,Unnamed: 0,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,0.747821,...,34.636410,367.041992,323.915100,-0.574928,0.356798,0.835559,87.140625,-103.895569,65000,1.0
1,1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,0.747821,...,34.636410,365.072906,323.915100,-0.570188,0.329104,0.835559,86.882576,-102.054672,65000,1.0
2,2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,0.747821,...,34.779556,363.663788,323.908203,-0.574245,0.340543,0.835559,86.781761,-89.001900,65000,1.0
3,3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,0.747821,...,39.965164,363.218719,202.661942,-0.586315,0.376283,0.835559,97.926498,-61.659561,65000,1.0
4,4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,0.747821,...,57.483765,363.460358,202.621964,-0.597914,0.407408,0.835559,74.817230,-33.565384,65000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134897,1170506,7/1-2 S,3169.3124,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.423170,4.768320,...,27.674368,156.274643,-0.068708,-0.001763,4.014674,3.345179,26.673708,4.078610,30000,2.0
1134898,1170507,7/1-2 S,3169.4644,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.379244,5.263275,...,28.024338,153.811325,-0.063919,-0.007600,3.139378,5.532522,26.840818,17.060884,65030,2.0
1134899,1170508,7/1-2 S,3169.6164,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.350248,11.335051,...,28.091282,154.413147,-0.156670,-0.018297,0.684578,13.706476,27.007942,36.792507,65030,2.0
1134900,1170509,7/1-2 S,3169.7684,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.313779,6.860022,...,28.019775,156.868408,-0.205981,-0.011438,2.349154,7.345001,27.175179,17.207531,65030,2.0


In [4]:
data_high = data[data['FORCE_2020_LITHOFACIES_CONFIDENCE']!=3]
data_high

,Unnamed: 0,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,0.747821,...,34.636410,367.041992,323.915100,-0.574928,0.356798,0.835559,87.140625,-103.895569,65000,1.0
1,1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,0.747821,...,34.636410,365.072906,323.915100,-0.570188,0.329104,0.835559,86.882576,-102.054672,65000,1.0
2,2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,0.747821,...,34.779556,363.663788,323.908203,-0.574245,0.340543,0.835559,86.781761,-89.001900,65000,1.0
3,3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,0.747821,...,39.965164,363.218719,202.661942,-0.586315,0.376283,0.835559,97.926498,-61.659561,65000,1.0
4,4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,0.747821,...,57.483765,363.460358,202.621964,-0.597914,0.407408,0.835559,74.817230,-33.565384,65000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134897,1170506,7/1-2 S,3169.3124,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.423170,4.768320,...,27.674368,156.274643,-0.068708,-0.001763,4.014674,3.345179,26.673708,4.078610,30000,2.0
1134898,1170507,7/1-2 S,3169.4644,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.379244,5.263275,...,28.024338,153.811325,-0.063919,-0.007600,3.139378,5.532522,26.840818,17.060884,65030,2.0
1134899,1170508,7/1-2 S,3169.6164,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.350248,11.335051,...,28.091282,154.413147,-0.156670,-0.018297,0.684578,13.706476,27.007942,36.792507,65030,2.0
1134900,1170509,7/1-2 S,3169.7684,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.313779,6.860022,...,28.019775,156.868408,-0.205981,-0.011438,2.349154,7.345001,27.175179,17.207531,65030,2.0


In [5]:
data_high['GROUP'].value_counts()

HORDALAND GP.       285680
SHETLAND GP.        231256
VIKING GP.          131815
DUNLIN GP.          118558
ROGALAND GP.        111959
NORDLAND GP.        107953
CROMER KNOLL GP.     52302
BAAT GP.             35770
VESTLAND GP.         26070
HEGRE GP.            13748
ZECHSTEIN GP.        11779
BOKNFJORD GP.         3125
ROTLIEGENDES GP.      2404
TYNE GP.              1205
Name: GROUP, dtype: int64

In [6]:
data_high['GROUP'].isnull().values.sum()

1278

In [7]:
formation =data_high['FORMATION'].tolist()

In [8]:
data_high['RMED'].isna().sum()

0

## Removing outliers

In [9]:
data_high=low_pass_filter_anomaly_detection(df=data_high, 
                                      column_name='CALI', 
                                      number_of_stdevs_away_from_mean=3)

In [10]:
data_high = data_high[data_high['CALI_Low_Pass_Filter_Anomaly'] == False]

In [11]:
data_high=low_pass_filter_anomaly_detection(df=data_high, 
                                      column_name='GR', 
                                      number_of_stdevs_away_from_mean=3)

C:\Users\Максимилиан\Desktop\Данные научка\Python files\FORCE COMPETITION\functions.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name+'_Rolling_Average']=df[column_name].rolling(window=60, center=True).mean()
C:\Users\Максимилиан\Desktop\Данные научка\Python files\FORCE COMPETITION\functions.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name+'_Rolling_StDev']=df[column_name].rolling(window=60, center=True).std()
C:\Users\Максимилиан\Desktop\Данные научка\Python files\FORCE

In [12]:
data_high = data_high[data_high['GR_Low_Pass_Filter_Anomaly'] == False]

In [13]:
data_high=low_pass_filter_anomaly_detection(df=data_high, 
                                      column_name='NPHI', 
                                      number_of_stdevs_away_from_mean=3)

In [14]:
data_high = data_high[data_high['NPHI_Low_Pass_Filter_Anomaly'] == False]

In [15]:
data_high=low_pass_filter_anomaly_detection(df=data_high, 
                                      column_name='RMED', 
                                      number_of_stdevs_away_from_mean=3)

In [16]:
data_high = data_high[data_high['RMED_Low_Pass_Filter_Anomaly'] == False]

In [17]:
data_high=low_pass_filter_anomaly_detection(df=data_high, 
                                      column_name='RHOB', 
                                      number_of_stdevs_away_from_mean=3)

In [18]:
data_high = data_high[data_high['RHOB_Low_Pass_Filter_Anomaly'] == False]

In [19]:
data_high=low_pass_filter_anomaly_detection(df=data_high, 
                                      column_name='DTC', 
                                      number_of_stdevs_away_from_mean=3)

In [20]:
data_high = data_high[data_high['DTC_Low_Pass_Filter_Anomaly'] == False]

In [21]:
data_high=low_pass_filter_anomaly_detection(df=data_high, 
                                      column_name='RDEP', 
                                      number_of_stdevs_away_from_mean=3)

In [22]:
data_high = data_high[data_high['RDEP_Low_Pass_Filter_Anomaly'] == False]

In [23]:
data_high=low_pass_filter_anomaly_detection(df=data_high, 
                                      column_name='ROP', 
                                      number_of_stdevs_away_from_mean=3)

In [24]:
data_high = data_high[data_high['ROP_Low_Pass_Filter_Anomaly'] == False]

In [25]:
data_high

,Unnamed: 0,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,...,RHOB_Low_Pass_Filter_Anomaly,DTC_Rolling_Average,DTC_Rolling_StDev,DTC_Low_Pass_Filter_Anomaly,RDEP_Rolling_Average,RDEP_Rolling_StDev,RDEP_Low_Pass_Filter_Anomaly,ROP_Rolling_Average,ROP_Rolling_StDev,ROP_Low_Pass_Filter_Anomaly
0,0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,0.747821,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False
1,1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,0.747821,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False
2,2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,0.747821,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False
3,3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,0.747821,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False
4,4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,0.747821,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134897,1170506,7/1-2 S,3169.3124,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.423170,4.768320,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False
1134898,1170507,7/1-2 S,3169.4644,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.379244,5.263275,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False
1134899,1170508,7/1-2 S,3169.6164,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.350248,11.335051,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False
1134900,1170509,7/1-2 S,3169.7684,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.313779,6.860022,...,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False


In [26]:
y = data_high['FORCE_2020_LITHOFACIES_LITHOLOGY']

lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11}
y = y.map(lithology_numbers)

In [27]:
def dropseq(grads, thresh = 10**(-6), seqthresh = 3):
    
    #pass grads as np array
    sequences = []
    print(grads)
    print(grads.T[0])
    j = 0
    for col in grads.T:
        streak = 1
        j+=1
        print(f'{j} out of {grads.T.shape[0]}')
        colstr = [1]
        print(col)
        for i in range(1, col.shape[0]):
            
            if abs(col[i]-col[i-1])<thresh:
                streak+=1
                colstr.append(np.nan)
            else:
                colstr.append(streak)
                streak = 1
                colstr = np.asarray(colstr) 
                sequences.append(colstr)
                print(colstr)
                print(sequences)
                seqdf = pd.DataFrame(data = sequences)
                print(seqdf)
                seqdf.fillna(inplace = True, method = 'bfill')
                a = seqdf.values
                print(a)
                a = np.where(a<seqthresh, 1, 0)
                print('finish')
        return a #multiply each col by its respective mask (here mask is a)

In [28]:
## Filling missing location data
well = data_high['WELL'].values
depth = data_high['DEPTH_MD'].values

X = data_high[['CALI','RMED','RDEP','GR','DEPTH_MD','DTC','ROP','RHOB','SP','DRHO','NPHI','PEF','BS']]

imp = SimpleImputer(strategy='mean')
imp.fit(X)
X = imp.transform(X)

augment_features_gradient(X, depth)

(array([[-7.91800658e-02,  4.38188250e-02, -2.00004954e-02, ...,
          1.70461441e-03, -1.00819437e+01,  2.59901348e-01],
        [ 0.00000000e+00,  5.51860586e-02,  3.35001632e-02, ...,
         -3.11151931e-02,  2.11085897e+01, -6.13614133e-03],
        [-6.26162500e-02, -3.20029888e-02,  5.15344868e-03, ...,
         -3.64686313e-04,  6.31604697e+01,  6.85181743e+00],
        ...,
        [-2.39930655e-01, -2.07078884e+02, -3.18563926e+00, ...,
         -5.09213852e-02, -9.59634781e-01,  0.00000000e+00],
        [-1.24134516e-01,  1.53269956e+01,  3.03105304e-02, ...,
          7.10765782e-03,  1.61925115e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 array([[ 5.20921485e-01,  7.47844313e-02,  3.51978017e-01, ...,
         -2.15919786e-01,  2.05200878e+02, -1.75024664e+00],
        [-4.11949013e-01, -5.73612154e-01, -1.86491543e-01, ...,
          2.02305966e-01,  2.76657105e

## Define number of wells for clusterisation

<b> Clusterisation of wells was considered as additional feature for classification, but it didn't bring good results. Anyway, this idea can make sense in future works <b>

In [29]:
wells = data['WELL'].unique()
pruned_data = data.drop_duplicates(subset = 'WELL', keep='first')[['WELL', 'X_LOC', 'Y_LOC']].dropna()
wells = pruned_data['WELL'].unique()
pruned_data

,WELL,X_LOC,Y_LOC
0,15/9-13,437641.96875,6470972.5
17913,15/9-15,436818.03125,6462991.5
34881,15/9-17,438591.96875,6478951.5
51454,16/1-2,455221.34375,6533321.5
53188,16/1-6 A,459499.68750,6539423.0
...,...,...,...
1085652,35/9-2,550223.68750,6800645.0
1095051,35/9-5,543505.68750,6799385.5
1099993,35/9-6 S,537126.50000,6804497.0
1113067,36/7-3,556075.18750,6810860.0


In [30]:
coords = np.empty(0)
coords2=[]
for well in wells:
    a = pruned_data.loc[pruned_data['WELL'] == well].X_LOC.item()
    b = pruned_data.loc[pruned_data['WELL'] == well].Y_LOC.item()
    c = np.array([a,b])
    coords2.append([a,b])
    coords = np.append(coords, c)
print(coords.shape)
print(coords2)
print(np.array(coords2).shape)
coords = np.array(coords2)
coords

(184,)
[[437641.96875, 6470972.5], [436818.03125, 6462991.5], [438591.96875, 6478951.5], [455221.34375, 6533321.5], [459499.6875, 6539423.0], [444152.125, 6435549.0], [443276.78125, 6444984.5], [460417.0, 6453752.5], [448701.6875, 6429298.5], [474566.5, 6436486.0], [475279.15625, 6518414.0], [476768.09375, 6523590.0], [477772.78125, 6520634.0], [449950.6875, 6500259.0], [474410.78125, 6510408.5], [442820.6875, 6461117.5], [446273.84375, 6468387.5], [466862.1875, 6479873.0], [520153.1875, 6452287.5], [470523.375, 6560848.5], [465848.8125, 6564135.5], [463756.0, 6562935.0], [469334.625, 6628720.0], [476936.15625, 6625441.5], [479890.5625, 6641111.5], [487292.09375, 6643944.0], [454469.53125, 6603730.0], [475844.90625, 6621548.0], [488713.125, 6598750.0], [489824.125, 6602677.5], [495697.25, 6618248.0], [454810.25, 6571041.0], [463972.28125, 6591330.0], [483550.6875, 6590310.0], [501149.5625, 6608191.5], [444585.59375, 6711937.5], [497360.46875, 6737311.5], [496651.78125, 6739799.5], [497

array([[ 437641.96875, 6470972.5    ],
       [ 436818.03125, 6462991.5    ],
       [ 438591.96875, 6478951.5    ],
       [ 455221.34375, 6533321.5    ],
       [ 459499.6875 , 6539423.     ],
       [ 444152.125  , 6435549.     ],
       [ 443276.78125, 6444984.5    ],
       [ 460417.     , 6453752.5    ],
       [ 448701.6875 , 6429298.5    ],
       [ 474566.5    , 6436486.     ],
       [ 475279.15625, 6518414.     ],
       [ 476768.09375, 6523590.     ],
       [ 477772.78125, 6520634.     ],
       [ 449950.6875 , 6500259.     ],
       [ 474410.78125, 6510408.5    ],
       [ 442820.6875 , 6461117.5    ],
       [ 446273.84375, 6468387.5    ],
       [ 466862.1875 , 6479873.     ],
       [ 520153.1875 , 6452287.5    ],
       [ 470523.375  , 6560848.5    ],
       [ 465848.8125 , 6564135.5    ],
       [ 463756.     , 6562935.     ],
       [ 469334.625  , 6628720.     ],
       [ 476936.15625, 6625441.5    ],
       [ 479890.5625 , 6641111.5    ],
       [ 487292.09375, 66

In [50]:
def Clusterisation(data):
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imput = imputer.fit(data[['X_LOC', 'Y_LOC']])
    data[['X_LOC', 'Y_LOC']] = imput.transform(data[['X_LOC', 'Y_LOC']])
    X = data['CLUSTERS'] = kmcluster.predict(data[['X_LOC', 'Y_LOC']])
    return X

In [51]:
kmcluster = sklearn.cluster.KMeans(n_clusters=4).fit(coords)

In [52]:
Clusterisation(data_high)

array([3, 3, 3, ..., 1, 1, 1])

In [ ]:
def numeric_preproccessing(data):
    num_pipeline = Pipeline([
        (('scaler', RobustScaler()))
    ])

    ## cathegorical encoding
    le = LabelEncoder()
    formation = data['FORMATION'].tolist()
    formation = le.fit_transform(formation)
    group = data['GROUP'].tolist()
    group = le.fit_transform(group)
    ## deleting outliers

    # transform needed features
    formation = num_pipeline.fit_transform(formation.reshape(-1, 1))
    data['CALI'] = num_pipeline.fit_transform(data['CALI'].values.reshape(-1, 1))
    data['RMED'] = num_pipeline.fit_transform(data['RMED'].values.reshape(-1, 1))
    data['RDEP'] = num_pipeline.fit_transform(data['RDEP'].values.reshape(-1, 1))
    data['GR'] = num_pipeline.fit_transform(data['GR'].values.reshape(-1, 1))
    data['DEPTH_MD'] = num_pipeline.fit_transform(data['DEPTH_MD'].values.reshape(-1, 1))
    data['DTC'] = num_pipeline.fit_transform(data['DTC'].values.reshape(-1, 1))
    data['ROP'] = num_pipeline.fit_transform(data['ROP'].values.reshape(-1, 1))
    data['RHOB'] = num_pipeline.fit_transform(data['RHOB'].values.reshape(-1, 1))
    data['SP'] = num_pipeline.fit_transform(data['SP'].values.reshape(-1, 1))
    data['DRHO'] = num_pipeline.fit_transform(data['DRHO'].values.reshape(-1, 1))
    data['NPHI'] = num_pipeline.fit_transform(data['NPHI'].values.reshape(-1, 1))
    data['PEF'] = num_pipeline.fit_transform(data['PEF'].values.reshape(-1, 1))
    data['BS'] = num_pipeline.fit_transform(data['BS'].values.reshape(-1, 1))

    data['DTS'] = num_pipeline.fit_transform(data['DTS'].values.reshape(-1, 1))

    Z = data[['CALI', 'RMED', 'RDEP', 'GR', 'DTC', 'RHOB', 'DRHO', 'NPHI']]
    gr1, gr2 = augment_features_gradient(Z, data['DEPTH_MD'])

    # imputer
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imput = imputer.fit(data[['X_LOC', 'Y_LOC']])
    data[['X_LOC', 'Y_LOC']] = imput.transform(data[['X_LOC', 'Y_LOC']])

    X = np.hstack((Z.values, gr1, gr2, data['DEPTH_MD'].values.reshape(-1,1)))

    return X

In [83]:
lll = numeric_preproccessing(data_high)

In [84]:
X = numeric_preproccessing(data_high)

In [85]:
X.shape

(1087344, 25)

## Tuning hyperparameters of the algorithm

In [40]:
# clf =OneVsOneClassifier(XGBClassifier(tree_method='gpu_hist'))


# param_grid = {'estimator__learning_rate': [0.1],
#              'estimator__n_estimators':[60],
#              'estimator__subsample': [0.3],
#            'estimator__max_depth': [6],
#              'estimator__tree_method': ['gpu_hist'],
#              'estimator__objective': ['binary:logistic'],
#              'estimator__max_delta_step': [4],
#              'estimator__reg_lambda': [1],
#              'estimator__min_child_weight': [6],
#              'estimator__colsample_bytree': [0.5]
#       }
# model_tuning = GridSearchCV(clf, param_grid, scoring ='f1_weighted', cv=3,verbose=10)

# model_tuning.fit(X,y)




# clf =OneVsOneClassifier(XGBClassifier(learning_rate=0.1,
#                                      n_estimators =80,
#                                      subsample =0.9,
#                                      max_depth =6,
#                                      tree_method='gpu_hist',
#                                      objective='binary:logistic',
#                                      gamma=0.1,
#                                      reg_alpha=0.5,  max_delta_step=2, min_child_weight=6, colsample_bytree=0.9))
# clf.fit(X,y)

In [41]:
 # model_tuning.best_estimator_

In [42]:
# model_tuning.best_params_

## Model

In [ ]:
clf =OneVsOneClassifier(XGBClassifier(learning_rate=0.1,
                                      n_estimators =60,
                                      subsample =0.3,
                                     max_depth =6,
                                      tree_method='gpu_hist',
                                      objective='binary:logistic',
                                      gamma=0.1,
                                      reg_alpha=1,  max_delta_step=4, min_child_weight=6, colsample_bytree=0.5,reg_lambda=1))
clf.fit(X,y)

In [54]:
# dump(clf, 'Model(17_10).joblib') 

NameError: name 'clf' is not defined

In [55]:
# clf = load('Model(17_10).joblib')

## Make prediction

In [56]:
data_high['WELL'].unique()

array(['15/9-13', '15/9-15', '15/9-17', '16/1-2', '16/1-6 A', '16/10-1',
       '16/10-2', '16/10-3', '16/10-5', '16/11-1 ST3', '16/2-11 A',
       '16/2-16', '16/2-6', '16/4-1', '16/5-3', '16/7-4', '16/7-5',
       '16/8-1', '17/11-1', '25/11-15', '25/11-19 S', '25/11-5',
       '25/2-13 T4', '25/2-14', '25/2-7', '25/3-1', '25/4-5', '25/5-1',
       '25/5-4', '25/6-1', '25/6-2', '25/6-3', '25/7-2', '25/8-5 S',
       '25/8-7', '25/9-1', '26/4-1', '29/6-1', '30/3-3', '30/3-5 S',
       '30/6-5', '31/2-1', '31/2-19 S', '31/2-7', '31/2-8', '31/2-9',
       '31/3-1', '31/3-2', '31/3-3', '31/3-4', '31/4-10', '31/4-5',
       '31/5-4 S', '31/6-5', '31/6-8', '32/2-1', '33/5-2', '33/6-3 S',
       '33/9-1', '33/9-17', '34/10-19', '34/10-21', '34/10-33',
       '34/10-35', '34/11-1', '34/11-2 S', '34/12-1', '34/2-4',
       '34/3-1 A', '34/4-10 R', '34/5-1 A', '34/5-1 S', '34/7-13',
       '34/7-20', '34/7-21', '34/8-1', '34/8-3', '34/8-7 R', '35/11-1',
       '35/11-10', '35/11-11', '35/11-12

In [57]:
test_data = data_high.loc[data_high['WELL']=='30/6-5']

In [58]:
test_data.append(data_high.loc[data_high['WELL']=='34/11-1'])

,Unnamed: 0,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,...,DTC_Rolling_Average,DTC_Rolling_StDev,DTC_Low_Pass_Filter_Anomaly,RDEP_Rolling_Average,RDEP_Rolling_StDev,RDEP_Low_Pass_Filter_Anomaly,ROP_Rolling_Average,ROP_Rolling_StDev,ROP_Low_Pass_Filter_Anomaly,CLUSTERS
505918,517760,30/6-5,257.2264,497437.37500,6728312.0,-232.226303,NORDLAND GP.,NaN,17.702616,2.133782,...,124.520522,43.006233,False,3.564182,3.464090,False,69.024980,61.127376,False,0
505919,517761,30/6-5,257.3784,497437.37500,6728312.0,-232.378311,NORDLAND GP.,NaN,16.079689,2.224826,...,126.097966,42.822118,False,3.109068,0.867109,False,69.965773,60.762688,False,0
505920,517762,30/6-5,257.5304,497437.37500,6728312.0,-232.530304,NORDLAND GP.,NaN,14.906407,2.308409,...,127.528544,42.753829,False,3.058264,0.858610,False,70.955697,60.396817,False,0
505921,517763,30/6-5,257.6824,497437.37500,6728312.0,-232.682312,NORDLAND GP.,NaN,14.490477,2.489255,...,128.955491,42.639708,False,3.008433,0.855767,False,72.878462,60.529943,False,0
505922,517764,30/6-5,257.8344,497437.37500,6728312.0,-232.834305,NORDLAND GP.,NaN,13.761547,2.553177,...,130.382988,42.477050,False,2.958714,0.855518,False,74.169511,60.175289,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794961,815650,34/11-1,4563.7070,473399.06250,6771833.5,-4538.615234,DUNLIN GP.,Statfjord Fm.,9.896898,6.732695,...,113.933562,34.490277,False,3.426877,1.865760,False,361.578377,424.816543,False,2
794962,815651,34/11-1,4563.8590,473399.06250,6771833.5,-4538.767090,DUNLIN GP.,Statfjord Fm.,9.886607,6.665358,...,115.098415,34.679191,False,3.384184,1.884744,False,372.252723,423.228559,False,2
794963,815652,34/11-1,4564.0110,473399.09375,6771833.5,-4538.918945,DUNLIN GP.,Statfjord Fm.,9.891300,6.559508,...,116.259569,34.831328,False,3.341528,1.903576,False,382.542363,421.718427,False,2
794964,815653,34/11-1,4564.1630,473399.09375,6771833.5,-4539.070801,DUNLIN GP.,Statfjord Fm.,9.899403,6.547416,...,117.410257,34.972350,False,3.300751,1.921238,False,392.665478,419.999619,False,2


In [59]:
test_data.append(data_high.loc[data_high['WELL']=='35/9-5'])
test_data.append(data_high.loc[data_high['WELL']=='31/4-5'])

test_data.append(data_high.loc[data_high['WELL']=='25/2-7'])

,Unnamed: 0,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,...,DTC_Rolling_Average,DTC_Rolling_StDev,DTC_Low_Pass_Filter_Anomaly,RDEP_Rolling_Average,RDEP_Rolling_StDev,RDEP_Low_Pass_Filter_Anomaly,ROP_Rolling_Average,ROP_Rolling_StDev,ROP_Low_Pass_Filter_Anomaly,CLUSTERS
505918,517760,30/6-5,257.226400,497437.37500,6728312.0,-232.226303,NORDLAND GP.,NaN,17.702616,2.133782,...,124.520522,43.006233,False,3.564182,3.464090,False,69.024980,61.127376,False,0
505919,517761,30/6-5,257.378400,497437.37500,6728312.0,-232.378311,NORDLAND GP.,NaN,16.079689,2.224826,...,126.097966,42.822118,False,3.109068,0.867109,False,69.965773,60.762688,False,0
505920,517762,30/6-5,257.530400,497437.37500,6728312.0,-232.530304,NORDLAND GP.,NaN,14.906407,2.308409,...,127.528544,42.753829,False,3.058264,0.858610,False,70.955697,60.396817,False,0
505921,517763,30/6-5,257.682400,497437.37500,6728312.0,-232.682312,NORDLAND GP.,NaN,14.490477,2.489255,...,128.955491,42.639708,False,3.008433,0.855767,False,72.878462,60.529943,False,0
505922,517764,30/6-5,257.834400,497437.37500,6728312.0,-232.834305,NORDLAND GP.,NaN,13.761547,2.553177,...,130.382988,42.477050,False,2.958714,0.855518,False,74.169511,60.175289,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280567,288170,25/2-7,4019.490001,479837.62500,6641080.5,-3993.574463,DUNLIN GP.,Sleipner Fm.,10.388752,4.725739,...,101.515631,19.951480,False,2.561916,1.127399,False,12.137455,8.228947,False,1
280568,288171,25/2-7,4019.642001,479837.59375,6641080.5,-3993.726318,DUNLIN GP.,Sleipner Fm.,10.602076,4.781342,...,102.318665,20.293504,False,2.520055,1.128308,False,11.837889,8.320314,False,1
280569,288172,25/2-7,4019.794001,479837.59375,6641080.5,-3993.878174,DUNLIN GP.,Sleipner Fm.,10.624195,4.785402,...,103.143694,20.613231,False,2.478384,1.127898,False,11.538398,8.399731,False,1
280570,288173,25/2-7,4019.946001,479837.59375,6641080.5,-3994.030029,DUNLIN GP.,Sleipner Fm.,10.643840,4.567891,...,103.963022,20.860979,False,2.437353,1.126675,False,11.253291,8.449844,False,1


In [60]:
open_test_features = test_data

In [61]:
# open_test_features = pd.read_csv('test.csv', sep=';')

In [62]:
Clusterisation(open_test_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Максимилиан\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

array([0, 0, 0, ..., 0, 0, 0])

In [63]:
open_test_features['FORMATION']=open_test_features['FORMATION'].fillna('Utsira Fm.')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Test missing data imputation

###  RMED prediction

In [64]:
RMEDpred = load('rmed.joblib')

In [65]:
xpred = open_test_features[pd.isna(open_test_features['RMED'])][['GR','RDEP']].values.reshape(-1, 2)
st =StandardScaler()

ypred = RMEDpred.predict(xpred)

In [66]:
open_test_features.loc[open_test_features['RMED'].isnull(), 'RMED'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['RMED'])].index)

C:\Users\Максимилиан\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


### RDEP prediction

In [67]:
RDEPpred = load('rdep.joblib')

In [68]:
xpred = open_test_features[pd.isna(open_test_features['RDEP'])]['GR'].values.reshape(-1, 1)

In [69]:
ypred = RDEPpred.predict(xpred)

In [70]:
open_test_features.loc[open_test_features['RDEP'].isnull(), 'RDEP'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['RDEP'])].index)

### DTC prediction

In [71]:
DTCpred = load('dtc.joblib')

In [72]:
xpred = open_test_features[pd.isna(open_test_features['DTC'])][['GR','RDEP','RMED']].values.reshape(-1, 3)
ypred = DTCpred.predict(xpred)

In [73]:
open_test_features.loc[open_test_features['DTC'].isnull(), 'DTC'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['DTC'])].index)

### Cali prediction

In [74]:
CALIpred = load('cali.joblib')

In [75]:
xpred = open_test_features[pd.isna(open_test_features['CALI'])][['GR','RDEP','RMED','DTC']].values.reshape(-1, 4)
ypred = CALIpred.predict(xpred)

In [76]:
open_test_features.loc[open_test_features['CALI'].isnull(), 'CALI'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['CALI'])].index)

### RHOB prediction

In [77]:
RHOBpred = load('rhob.joblib')

In [78]:
xpred = open_test_features[pd.isna(open_test_features['RHOB'])][['GR','RDEP','RMED','DTC','CALI']].values.reshape(-1, 5)
ypred = RHOBpred.predict(xpred)

In [79]:
open_test_features.loc[open_test_features['RHOB'].isnull(), 'RHOB'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['RHOB'])].index)

### DRHO prediction

In [80]:
DRHOpred = load('DRHO.joblib')

In [81]:
xpred = open_test_features[pd.isna(open_test_features['DRHO'])][['GR','RDEP','RMED','DTC','CALI','RHOB']].values.reshape(-1, 6)
ypred = DRHOpred.predict(xpred)

In [82]:
open_test_features.loc[open_test_features['DRHO'].isnull(), 'DRHO'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['DRHO'])].index)

### SP prediction

In [83]:
SPpred = load('sp.joblib')

In [84]:
xpred = open_test_features[pd.isna(open_test_features['SP'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO']].values.reshape(-1, 7)
ypred = SPpred.predict(xpred)

In [85]:
open_test_features.loc[open_test_features['SP'].isnull(), 'SP'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['SP'])].index)

### NPHI predicton

In [86]:
NPHIpred = load('nphi.joblib')

In [87]:
xpred = open_test_features[pd.isna(open_test_features['NPHI'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP']].values.reshape(-1, 8)
ypred = NPHIpred.predict(xpred)

In [88]:
open_test_features.loc[open_test_features['NPHI'].isnull(), 'NPHI'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['NPHI'])].index)

### PEF prediction

In [89]:
PEFpred = load('pef.joblib')

In [91]:
xpred = open_test_features[pd.isna(open_test_features['PEF'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI']].values.reshape(-1, 9)
ypred = PEFpred.predict(xpred)

In [92]:
open_test_features.loc[open_test_features['PEF'].isnull(), 'PEF'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['PEF'])].index)

C:\Users\Максимилиан\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


### RSHA prediction

In [93]:
RSHApred = load('rsha.joblib')

In [94]:
xpred = open_test_features[pd.isna(open_test_features['RSHA'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI','PEF']].values.reshape(-1, 10)
ypred = RSHApred.predict(xpred)

In [95]:
open_test_features.loc[open_test_features['RSHA'].isnull(), 'RSHA'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['RSHA'])].index)

### RXO prediction

In [96]:
RXOpred = load('rxo.joblib')

In [97]:
xpred = open_test_features[pd.isna(open_test_features['RXO'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI','PEF','RSHA']].values.reshape(-1, 11)
ypred = RXOpred.predict(xpred)

In [98]:
open_test_features.loc[open_test_features['RXO'].isnull(), 'RXO'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['RXO'])].index)

### BS prediction

In [99]:
BSpred = load('bs.joblib')

In [100]:
xpred = open_test_features[pd.isna(open_test_features['BS'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI','PEF','RSHA','RXO']].values.reshape(-1, 12)
ypred = BSpred.predict(xpred)

In [101]:
open_test_features.loc[open_test_features['BS'].isnull(), 'BS'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['BS'])].index)

### DCAL prediction

In [102]:
DCALpred = load('dcal.joblib')

In [103]:
xpred = open_test_features[pd.isna(open_test_features['DCAL'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI','PEF','RSHA','RXO','BS']].values.reshape(-1, 13)
ypred = DCALpred.predict(xpred)

In [104]:
open_test_features.loc[open_test_features['DCAL'].isnull(), 'DCAL'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['DCAL'])].index)

### ROP prediction

In [105]:
ROPpred = load('rop.joblib')

In [106]:
xpred = open_test_features[pd.isna(open_test_features['ROP'])][['GR', 'RDEP', 'RMED', 'DTC', 'CALI', 'RHOB', 'DRHO', 'SP', 'NPHI', 'PEF', 'RSHA', 'RXO', "BS", 'DCAL']].values.reshape(-1, 14)
ypred = ROPpred.predict(xpred)

In [107]:
open_test_features.loc[open_test_features['ROP'].isnull(), 'ROP'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['ROP'])].index)

### MUDWEIGHT prediction

In [108]:
MUDpred = load('mudweight.joblib')

In [109]:
xpred = open_test_features[pd.isna(open_test_features['MUDWEIGHT'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI','PEF','RSHA','RXO','BS','DCAL','ROP']].values.reshape(-1, 15)
ypred = MUDpred.predict(xpred)

In [110]:
open_test_features.loc[open_test_features['MUDWEIGHT'].isnull(), 'MUDWEIGHT'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['MUDWEIGHT'])].index)

### SGR prediction

In [111]:
SGRpred = load('sgr.joblib')

In [112]:
xpred = open_test_features[pd.isna(open_test_features['SGR'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI','PEF','RSHA','RXO','BS','DCAL','ROP','MUDWEIGHT']].values.reshape(-1, 16)
ypred = SGRpred.predict(xpred)

In [113]:
open_test_features.loc[open_test_features['SGR'].isnull(), 'SGR'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['SGR'])].index)

### RMIC prediction

In [114]:
RMICpred = load('rmic.joblib')

In [115]:
xpred = open_test_features[pd.isna(open_test_features['RMIC'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI','PEF','RSHA','RXO','BS','DCAL','ROP','MUDWEIGHT','SGR']].values.reshape(-1, 17)
ypred = RMICpred.predict(xpred)

In [116]:
open_test_features.loc[open_test_features['RMIC'].isnull(), 'RMIC'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['RMIC'])].index)

### DTS prediction

In [117]:
DTSpred = load('dts.joblib')

In [118]:
xpred = open_test_features[pd.isna(open_test_features['DTS'])][['GR','RDEP','RMED','DTC','CALI','RHOB','DRHO','SP','NPHI','PEF','RSHA','RXO','BS','DCAL','ROP','MUDWEIGHT','SGR','RMIC']].values.reshape(-1, 18)
ypred = DTSpred.predict(xpred)

In [119]:
open_test_features.loc[open_test_features['DTS'].isnull(), 'DTS'] = pd.Series(ypred, index = open_test_features[pd.isna(open_test_features['DTS'])].index)

In [120]:
X_test = numeric_preproccessing(open_test_features)

C:\Users\Максимилиан\Desktop\Данные научка\Python files\FORCE COMPETITION\functions.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CALI'] = num_pipeline.fit_transform(data['CALI'].values.reshape(-1, 1))
C:\Users\Максимилиан\Desktop\Данные научка\Python files\FORCE COMPETITION\functions.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['RMED'] = num_pipeline.fit_transform(data['RMED'].values.reshape(-1, 1))
C:\Users\Максимилиан\Desktop\Данные научка\Python files\FORCE COMPETITION\functi

In [121]:
X_test.shape

(20263, 25)

In [122]:
test_prediction =clf.predict(X_test)

In [123]:
test_prediction

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [126]:
test_prediction_for_submission = np.vectorize(category_to_lithology.get)(test_prediction)

NameError: name 'category_to_lithology' is not defined

In [204]:
test_prediction_for_submission

array([65000, 65000, 65000, ..., 65000, 65000, 65000])

In [185]:
f1_score(test_data['FORCE_2020_LITHOFACIES_LITHOLOGY'].values,test_prediction_for_submission,average= 'weighted')

0.7457630929890706

## Converting back

In [127]:
category_to_lithology = {y:x for x,y in lithology_numbers.items()}

In [128]:
test_prediction_for_submission = np.vectorize(category_to_lithology.get)(test_prediction)

In [129]:
np.savetxt('test_predictions17_10.csv', test_prediction_for_submission, header='lithology', comments='', fmt='%i')